In [93]:
import os,sys
sys.path.append('PigmentHunter/misc/lib/python3.7/site-packages')


from IPython.display import display
import math
import numpy as np
import nglview as nv
import matplotlib.pyplot as plt
import parmed as pmd
import numpy.random as random
import re
import requests
from hubzero.submit.SubmitCommand import SubmitCommand
import time
import hublib.tool as tool
import hublib.ui as ui
import ipywidgets as widgets


from IPython.display import Javascript, display


class ExcStruc:
    def __init__(self):
        self.nres = 0
        self.resnums = []
        self.restypes = []
        self.ham = []
        self.dips = []
        self.cents = []
            
    def reset(self):
        self.nres = 0
        self.resnums = []
        self.restypes = []
        self.ham = []
        self.dips = []
        self.cents = []

class RepList:
    def __init__(self):
        self.component_id = ''
        self.params = []
        self.names = []
        self.reps = list()
        
    def reset(self):
        self.component_id = ''
        self.params = []
        self.names = []
        self.reps = list()
        
    def append(self, nrep):
        self.names.append(nrep.name)
        ptext = {"type": nrep.type, "params": {"color": nrep.color, "sele": nrep.selection, "opacity": str(nrep.opacity)}}
        self.params.append(ptext)
        self.reps.append(nrep)
        
class NGLRep:
    def __init__(self, name, rtype, sel, col, opac):
        self.name = name
        self.type = rtype
        self.selection = sel
        self.color = col
        self.opacity = opac

def check_atoms(RefAtNames, QuerAtNames):
    ismatch = True
    for atnm in RefAtNames:
        if(QuerAtNames.count(atnm)!=1):
            ismatch = False
    return ismatch

def find_porph(pmdstruc, xtruc):
    PORatList = list(['NA', 'NB', 'NC', 'ND'])
    for n in range(0, len(pmdstruc.residues)):
        r = pmdstruc.residues[n]
        nmlist = list()
        for at in r:
            nmlist.append(at.name)
        if(check_atoms(PORatList, nmlist)):
            if(nmlist.count('MG')==1):
                xstruc.resnums.append(n)
                xstruc.nres += 1 
                xstruc.restypes.append('CHL')
                
            else:
                xstruc.resnums.append(n)
                xstruc.nres += 1 
                xstruc.restypes.append('PHO')

    return


def find_dipoles(struc,xtruc):
    ResNums = xtruc.resnums
    DipMat = np.zeros((len(ResNums),3))
    for n in range(0, len(ResNums)):
        r = struc.residues[ResNums[n]]
        for at in r:
            if at.name=='NB':
                NB = struc.coordinates[at.idx]
            if at.name=='ND':
                ND = struc.coordinates[at.idx]
        DipMat[n,:] = NB - ND
    xtruc.dips = DipMat

def find_centers(struc, xtruc):
    ResNums = xtruc.resnums
    CentMat = np.zeros((len(ResNums),3))
    for n in range(0, len(ResNums)):
        r = struc.residues[ResNums[n]]
        cent = 0.0
        for at in r:
            if at.name=='NB':
                cent += 0.5*struc.coordinates[at.idx]
            if at.name=='ND':
                cent += 0.5*struc.coordinates[at.idx]
        CentMat[n,:] = cent
    xtruc.cents = CentMat

def sync_widgets_to_rep(rep):
    seldrop.value = rep.name
    styledrop.value = rep.type
    colordrop.value = rep.color
    opacslide.value = int(rep.opacity*100)
    
def sync_rep_to_widgets():
    global rList
    num = rList.names.index(seldrop.value)
    rep = rList.reps[num]
    rep.type = styledrop.value
    rep.name = seldrop.value
    rep.color = colordrop.value
    rep.opacity = opacslide.value*0.01
    ptext = {"type": rep.type, "params": {"color": rep.color, "sele": rep.selection, "opacity": str(rep.opacity)}}
    rList.params[num] = ptext
    pdbview.set_representations(rList.params)
    

def std_rep(nglview, nvstruc, xstruc):
    global rList
    pdbview.add_trajectory(struc)
    pdbview.clear(0)
    chltxt = ''
    photxt = ''
    for n in range(0, len(xstruc.resnums)):
        if xstruc.restypes[n]=='CHL':
            if len(chltxt)>0:
                chltxt += ' or '
            chltxt += str(xstruc.resnums[n]+1)
        if xstruc.restypes[n]=='PHO':
            if len(photxt)>0:
                photxt += ' or '
            photxt += str(xstruc.resnums[n]+1)
    
    rList.append(NGLRep("Protein", "cartoon", "protein", "grey", 0.2))
    if len(chltxt)>0:
        rList.append(NGLRep("Chlorophyll", "licorice", chltxt, "green", 1.0))
    if len(photxt)>0:
        rList.append(NGLRep("Pheophytin", "licorice", photxt, "blue", 1.0))
    
    pdbview.set_representations(rList.params)
    
    seldrop.options=rList.names
    seldrop.disabled=False
    
    styledrop.disabled=False
    
    colordrop.disabled=False
    opacslide.disabled=False
    sync_widgets_to_rep(rList.reps[0])
    
    for n in range(0, np.shape(xstruc.dips)[0]):
        cent = xstruc.cents[n,:]
        dip = xstruc.dips[n,:]
        v1 = cent - 1.5*dip
        v2 = cent + 1.5*dip
        pdbview.shape.add_arrow(v1.tolist(), v2.tolist(), [1,0,0 ], 1.0 )
        
        
pdbid = widgets.Text(
    value='2DRE',
    placeholder='Type something',
    description='PDB ID:',
    layout = widgets.Layout(width='4cm'),
    disabled=False
)

pdbgo = widgets.Button(
    description='Display',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to display the pdb file',
    icon='' # (FontAwesome names without the `fa-` prefix)
)

pdboutput = widgets.HTML(
    value="",
    placeholder='',
    description='',
)

Output = widgets.Output()

def clear_stage(view):
    view._clear_component_auto_completion()
    if view._trajlist:
        for traj in view._trajlist:
            view._trajlist.remove(traj)
    for component_id in view._ngl_component_ids:
        component_index = view._ngl_component_ids.index(component_id)
        view._ngl_component_ids.remove(component_id)
        view._ngl_component_names.pop(component_index)
        view._remote_call('removeComponent',
            target='Stage',
            args=[component_index,])
    # Shapes appear not to be populated in component_id list.
    # So we have to call seperately removeAllComponents to 
    # get rid of them. 
    view._remote_call('removeAllComponents',
            target='Stage',
            args=[0])
    view._update_component_auto_completion()


def pdbgo_onclick(b):
    global struc
    global estruc
    url = 'http://files.rcsb.org/download/'+pdbid.value+'.pdb'
    r = requests.get(url, allow_redirects=True)
    if(r.status_code==200):
        fname = pdbid.value + '.pdb'
        wfd = open(fname, 'wb')
        wfd.write(r.content)
        wfd.close()
        pdboutput.value = ''
        
        xstruc.reset()
        clear_stage(pdbview)
        rList.reset()
        struc = pmd.load_file(fname)
        find_porph(struc, xstruc)
        find_dipoles(struc, xstruc)
        find_centers(struc, xstruc)
        std_rep(pdbview, struc, xstruc)

    else:
        pdboutput.value = 'Please enter a valide PDB ID code.'

        
xstruc = ExcStruc()
pdbgo.on_click(pdbgo_onclick)
pdbid.on_submit(pdbgo_onclick)
pdbview = nv.NGLWidget()
pdbview._set_size('500px', '500px')
pdbbox = widgets.HBox([pdbid, pdbgo])
rList = RepList()


seldrop = widgets.Dropdown(
    options=rList.names,
    #value='Protein',
    description='Selection:',
    disabled=True,
)


styledrop = widgets.Dropdown(
    options=['cartoon', 'licorice', 'spacefill'],
    description='Style:',
    disabled=True,
)

colordrop = widgets.Dropdown(
    options=['chain', 'red', 'green', 'blue', 'grey'],
    description='Color:',
    disabled=True,
)

opacslide = widgets.IntSlider(
    min=0,
    max=100,
    step=1,
    description='Opacity:',
    disabled=True,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

dispbox = widgets.VBox([seldrop, styledrop, colordrop, opacslide])
viewbox = widgets.HBox([pdbview, dispbox])


def seldrop_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        rep = rList.reps[rList.names.index(seldrop.value)]
        sync_widgets_to_rep(rep)
        
def styledrop_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        num = rList.names.index(seldrop.value)
        rep = rList.reps[num]
        rep.type = styledrop.value
        ptext = {"type": rep.type, "params": {"color": rep.color, "sele": rep.selection, "opacity": str(rep.opacity)}}
        rList.params[num] = ptext
        pdbview.set_representations(rList.params)
        
def colordrop_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        num = rList.names.index(seldrop.value)
        rep = rList.reps[num]
        rep.color = colordrop.value
        ptext = {"type": rep.type, "params": {"color": rep.color, "sele": rep.selection, "opacity": str(rep.opacity)}}
        rList.params[num] = ptext
        pdbview.set_representations(rList.params)
        
def opacslide_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        num = rList.names.index(seldrop.value)
        rep = rList.reps[num]
        rep.opacity = opacslide.value*0.01
        ptext = {"type": rep.type, "params": {"color": rep.color, "sele": rep.selection, "opacity": str(rep.opacity)}}
        rList.params[num] = ptext
        pdbview.set_representations(rList.params)

seldrop.observe(seldrop_on_change)
styledrop.observe(styledrop_on_change)
colordrop.observe(colordrop_on_change)
opacslide.observe(opacslide_on_change)
    
inprogress = False
        
display(pdbbox)
display(pdboutput)
display(viewbox)
struc = pmd.structure


HTML(value='', placeholder='')

In [87]:
def done_cb(w, name):
    # Do something with the files here...
    # We just print their names
    print("%s uploaded" % name)
    
    # reset clears and re-enables the widget for more uploads
    # You may want to do this somewhere else, depending on your GUI
    #w.reset()
PrNameui = ui.FileUpload("Protean File", 
               "PDB file to upload, example.pdb",
               dir='tmpdir',
               cb=done_cb,
               maxsize='10M')
display(PrNameui)

In [179]:
new_struc=[]
new_coord=[]
atomlist=pmd.structure.ResidueList(struc)
coordinates=pmd.Structure.get_coordinates(struc)
for n in range (0,len(atomlist)):
    if "HOH" not in str(atomlist[n]):
        if " N " in str(atomlist[n]):
            new_struc.append(atomlist[n])
            x=float(coordinates[0,n,0])
            y=float(coordinates[0,n,1])
            z=float(coordinates[0,n,2])
            new_coord.append([x,y,z])
        if " CA " in str(atomlist[n]):
            new_struc.append(atomlist[n])
            x=float(coordinates[0,n,0])
            y=float(coordinates[0,n,1])
            z=float(coordinates[0,n,2])
            new_coord.append([x,y,z])
        if " C " in str(atomlist[n]):
            new_struc.append(atomlist[n])
            x=float(coordinates[0,n,0])
            y=float(coordinates[0,n,1])
            z=float(coordinates[0,n,2])
            new_coord.append([x,y,z])      
        if " O " in str(atomlist[n]):
            new_struc.append(atomlist[n])
            x=float(coordinates[0,n,0])
            y=float(coordinates[0,n,1])
            z=float(coordinates[0,n,2])
            new_coord.append([x,y,z])



6168

In [180]:
#help(pmd.structure.AtomList)
#new_widget=nv.NGLWidget()
#filename="2dre"
#new_widget.add_pdbid(filename)
#display(new_widget)
#count=0
#for n in range (0, len (matrix)):
#    for m in range (count, len(matrix)):
#        ans=matrix(count)-matrix(m)
#        if ans /= 0:
#            trueans.append(ans)
#    count=count+1

In [51]:
help(pmd.structure.AtomList)

Help on class AtomList in module parmed.topologyobjects:

class AtomList(TrackedList)
 |  AtomList(*args)
 |  
 |  Array of Atoms
 |  
 |  Notes
 |  -----
 |  Deleting an atom from the AtomList also deletes that atom from the residue
 |  it belongs to.
 |  
 |  Method resolution order:
 |      AtomList
 |      TrackedList
 |      builtins.list
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __delitem__ = new_func(self, *args, **kwargs)
 |  
 |  __iadd__(self, other)
 |      Implement self+=value.
 |  
 |  append = new_func(self, *args, **kwargs)
 |  
 |  assign_nbidx_from_types(self)
 |      Assigns the nb_idx attribute of every atom inside here from the
 |      atom_type definition. If the atom_type is not assigned, RuntimeError is
 |      raised.
 |      
 |      Returns
 |      -------
 |      ``list of dict``
 |          Each element is a `set` of the `nb_idx` indices for which NBFIX
 |          alterations are defined for the type with that given that index
 |     

In [19]:
pdbid = widgets.Text(
    value='2DRE',
    placeholder='Type something',
    description='PDB ID:',
    layout = widgets.Layout(width='4cm'),
    disabled=False
)
display(pdbid)

Text(value='2DRE', description='PDB ID:', layout=Layout(width='4cm'), placeholder='Type something')

In [121]:
help(pmd.Structure.strip)

Help on function strip in module parmed.structure:

strip(self, selection)
    Deletes a subset of the atoms corresponding to an atom-based selection.
    
    Parameters
    ----------
    selection : :class:`AmberMask`, ``str``, or ``iterable``
        This is the selection of atoms that will be deleted from this
        structure. If it is a string, it will be interpreted as an
        AmberMask. If it is an AmberMask, it will be converted to a
        selection of atoms. If it is an iterable, it must be the same length
        as the `atoms` list.



In [137]:
help(pmd.Structure.get_coordinates)

Help on function get_coordinates in module parmed.structure:

get_coordinates(self, frame='all')
    In some cases, multiple conformations may be stored in the Structure.
    This function retrieves a particular frame's coordinates
    
    Parameters
    ----------
    frame : int or 'all', optional
        The frame number whose coordinates should be retrieved. Default is
        'all'
    
    Returns
    -------
    coords : np.ndarray, shape([#,] natom, 3) or None
        If frame is 'all', all coordinates are returned with shape
        (#, natom, 3). Otherwise the requested frame is returned with shape
        (natom, 3). If no coordinates exist and 'all' is requested, None is
        returned
    
    Raises
    ------
    IndexError if there are fewer than ``frame`` coordinates

